In [112]:
import websocket
import json
from urllib.parse import quote

In [113]:
ws = websocket.WebSocket()
website_id='google.com'
ws.connect(f'wss://plankton-app-5rfza.ondigitalocean.app/websites?scraping={website_id}')
ws.recv()

'{"id": "google.com", "scraping": true}'

In [114]:
ws.close()

In [117]:
ws = websocket.WebSocket()
page_id = 'https://betway.com/es/sports/evt/10647401'
website_id='betway.com'
uri = 'wss://plankton-app-5rfza.ondigitalocean.app'
ws.connect(f'{uri}/games/{website_id}?game_id={page_id}')
ws.recv()

ValueError: scheme fwss is invalid

In [47]:
ws.close()

In [72]:
from core.models import r
from redis.commands.search.query import Query, NumericFilter

In [108]:
r.ft('idxGames').search(Query('@scraping:{true}').paging(0, 5))

Result{0 total, docs: []}

In [104]:
from core.webdriver import init_driver
driver = init_driver(False)

In [105]:
driver.execute_script("window.open('');")